# jupyter notebook to calculate the optimal combination of weights for pm

In [1]:
import pandas as pd
from scipy.optimize import nnls
from sklearn.metrics import mean_squared_error

In [8]:
ranking = pd.read_csv('actual_rankings.csv')
#has 99 if the team wasnt playing that year cause in 2022 there was an expansion
#should replace with 0
ranking = ranking.replace(99, 0)
ranking.head()

,Team,2008 Actual Rank,2009 Actual Rank,2010 Actual Rank,2011 Actual Rank,2012 Actual Rank,2013 Actual Rank,2014 Actual Rank,2015 Actual Rank,2016 Actual Rank,2017 Actual Rank,2018 Actual Rank,2019 Actual Rank,2020 Actual Rank,2021 Actual Rank,2022 Actual Rank,2023 Actual Rank
0,Kolkata Night Riders,6,8,6,4,2,7,2,5,4,4,3,5,5,4,7,7
1,Mumbai Indians,5,7,1,3,3,2,4,2,5,1,5,1,1,5,10,4
2,Kings XI Punjab,2,5,8,5,6,6,1,8,8,5,7,6,6,6,6,8
3,Sunrisers Hyderbad,8,4,2,7,8,4,6,6,3,3,1,4,3,8,8,10
4,Rajasthan Royals,1,6,7,6,7,3,5,4,0,0,4,7,8,7,2,5


In [9]:
teams = ranking['Team'].tolist()
print(teams)

['Kolkata Night Riders', 'Mumbai Indians', 'Kings XI Punjab', 'Sunrisers Hyderbad', 'Rajasthan Royals', 'Delhi Capitals', 'Royal Challengers Bangalore', 'Chennai Super Kings', 'Gujarat Titans', 'Lucknow Super Giants', 'Kochi Tuskers Kerala']


In [10]:
predicted_rankings = pd.read_csv('predicted_rankings.csv')
if 'Unnamed: 0' in predicted_rankings.columns:
    predicted_rankings = predicted_rankings.drop(columns=['Unnamed: 0'])
predicted_rankings.head()

#now here have to insert rows for each team that wasnt there that year and give 0 aswell
#the names of these teams are the Lucknow Super Giants and Gujarat Titans, so insert rows for both team with rank 0 for each year

,team_name,rank_pm1,rank_pm2,rank_pm3,year
0,Chennai Super Kings,2,4,1,2008
1,Rajasthan Royals,4,2,3,2008
2,Royal Challengers Bangalore,8,8,6,2008
3,Kolkata Knight Riders,7,7,4,2008
4,Delhi Capitals,5,3,2,2008


In [11]:
years = predicted_rankings['year'].unique()
    
team_rt = "Rajasthan Royals"
team_csk = "Chennai Super Kings"
    
team_gt = "Gujarat Titans"
team_lsg = "Lucknow Super Giants"
team_ktk = "Kochi Tuskers Kerala"

"""for year in range(2008,2022):
    for team in new_teams:
        new_rows.append({
            'team_name': team,
            'rank_pm1': 0,
            'rank_pm2': 0,
            'rank_pm3': 0,
            'year': year
        })"""


new_rows = []
#Adding the disqualified teams only for the years they were nt disqualified
for year in range(2016, 2017):
    new_rows.append({
        'team_name': team_rt,
        'rank_pm1': 0,
        'rank_pm2': 0,
        'rank_pm3': 0,
        'year': year
    })
    new_rows.append({
        'team_name': team_csk,
        'rank_pm1': 0,
        'rank_pm2': 0,
        'rank_pm3': 0,
        'year': year
    })


#LSG did not exist between 08-10 and 14-15 and 17-21
for year in range(2008, 2010):
        new_rows.append({
        'team_name': team_lsg, 
        'rank_pm1': 0,
        'rank_pm2': 0,
        'rank_pm3': 0,
        'year': year
    })

for year in range(2014,2015):
        new_rows.append({
        'team_name': team_lsg,
        'rank_pm1': 0,
        'rank_pm2': 0,
        'rank_pm3': 0,
        'year': year
    })
# LSG and GT did not exist between 2018 and 2021
for year in range(2018,2021):
        new_rows.append({
        'team_name': team_lsg,
        'rank_pm1': 0,
        'rank_pm2': 0,
        'rank_pm3': 0,
        'year': year
    })
        new_rows.append({
        'team_name': team_gt,
        'rank_pm1': 0,
        'rank_pm2': 0,
        'rank_pm3': 0,
        'year': year
    })
        
# Gujarat Titans Inactive Years
for year in range(2008,2015):
        new_rows.append({
        'team_name': team_gt,
        'rank_pm1': 0,
        'rank_pm2': 0,
        'rank_pm3': 0,
        'year': year
    })
        

#Adding Kochi Tuskers Kerala for the year they were active
for year in range(2008, 2010):
    new_rows.append({
        'team_name': team_ktk,
        'rank_pm1': 0,
        'rank_pm2': 0,
        'rank_pm3': 0,
        'year': year
    })
    
for year in range(2012, 2023):
    new_rows.append({
        'team_name': team_ktk,
        'rank_pm1': 0,
        'rank_pm2': 0,
        'rank_pm3': 0,
        'year': year
    })

new_teams_df = pd.DataFrame(new_rows)

predicted_rankings = pd.concat([predicted_rankings, new_teams_df], ignore_index=True)

predicted_rankings.head()

,team_name,rank_pm1,rank_pm2,rank_pm3,year
0,Chennai Super Kings,2,4,1,2008
1,Rajasthan Royals,4,2,3,2008
2,Royal Challengers Bangalore,8,8,6,2008
3,Kolkata Knight Riders,7,7,4,2008
4,Delhi Capitals,5,3,2,2008


The reason I give zero to the ranking for the expansion teams is because I gave them actual rank 0 aswell. So whatever the weights, the loss will always be zero so they wont influence the LR but still give everything matching dimensions.

In [12]:
# Reshape actual ranking data for merging
ranking_melted = ranking.melt(id_vars=['Team'], 
                              value_vars=['2008 Actual Rank', '2009 Actual Rank', '2010 Actual Rank', '2011 Actual Rank', '2012 Actual Rank', '2013 Actual Rank', '2014 Actual Rank', '2015 Actual Rank', '2016 Actual Rank', '2017 Actual Rank', '2018 Actual Rank', '2019 Actual Rank', '2020 Actual Rank', '2021 Actual Rank', '2022 Actual Rank', '2023 Actual Rank'],
                              var_name='Year', value_name='Actual Rank')

# Extract the year from the 'Year' column
ranking_melted['Year'] = ranking_melted['Year'].str.extract('(\d+)').astype(int)

# Merge metrics rankings and actual rankings
data = pd.merge(predicted_rankings, ranking_melted, left_on=['team_name', 'year'], right_on=['Team', 'Year'])

# Drop unnecessary columns
data = data.drop(columns=['Team', 'year'])

print(data)

                       team_name  rank_pm1  rank_pm2  rank_pm3  Year  \
0            Chennai Super Kings         2         4         1  2008   
1               Rajasthan Royals         4         2         3  2008   
2    Royal Challengers Bangalore         8         8         6  2008   
3                 Delhi Capitals         5         3         2  2008   
4                Kings XI Punjab         1         1         5  2008   
..                           ...       ...       ...       ...   ...   
133         Kochi Tuskers Kerala         0         0         0  2018   
134         Kochi Tuskers Kerala         0         0         0  2019   
135         Kochi Tuskers Kerala         0         0         0  2020   
136         Kochi Tuskers Kerala         0         0         0  2021   
137         Kochi Tuskers Kerala         0         0         0  2022   

     Actual Rank  
0              3  
1              1  
2              7  
3              4  
4              2  
..           ...  
13

<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_9360\418766499.py:7: SyntaxWarning: invalid escape sequence '\d'
  ranking_melted['Year'] = ranking_melted['Year'].str.extract('(\d+)').astype(int)


In [13]:
X = data[['rank_pm1', 'rank_pm2', 'rank_pm3']]
y = data['Actual Rank']

# Use Non-Negative Least Squares (NNLS) to fit the model
weights, residual = nnls(X, y)

print("Weights: ", weights)
print("Residual: ", residual)

# Use the model to predict rankings
predicted_rankings = X.dot(weights)

# Evaluate the model
mse = mean_squared_error(y, predicted_rankings)
print("Mean Squared Error: ", mse)

# Print the actual and predicted rankings
result = data[['Year', 'Actual Rank', 'team_name']].copy()
result['Predicted Rank'] = predicted_rankings
print(result)



Weights:  [0.         0.72397467 0.25746868]
Residual:  20.152568811032005
Mean Squared Error:  2.942942244082463
     Year  Actual Rank                    team_name  Predicted Rank
0    2008            3          Chennai Super Kings        3.153367
1    2008            1             Rajasthan Royals        2.220355
2    2008            7  Royal Challengers Bangalore        7.336609
3    2008            4               Delhi Capitals        2.686861
4    2008            2              Kings XI Punjab        2.011318
..    ...          ...                          ...             ...
133  2018            0         Kochi Tuskers Kerala        0.000000
134  2019            0         Kochi Tuskers Kerala        0.000000
135  2020            0         Kochi Tuskers Kerala        0.000000
136  2021            0         Kochi Tuskers Kerala        0.000000
137  2022            0         Kochi Tuskers Kerala        0.000000

[138 rows x 4 columns]


In [14]:
import os
directory = 'C:/Users/Lenovo/OneDrive/Desktop/Performance Metric Phase III'
filename = 'LR_predicted_rankings.csv'
file_path = os.path.join(directory, filename)
result.to_csv(file_path)